# ContextualCompressionRetriever로 검색 결과 압축하기

이 튜토리얼에서는 LangChain의 `ContextualCompressionRetriever`를 사용하여 검색된 문서에서 질의와 관련된 정보만을 추출하는 방법을 학습합니다.

## 목차
1. [개요](#개요)
2. [기본 Retriever 설정](#기본-retriever-설정)
3. [맥락적 압축 (ContextualCompression)](#맥락적-압축-contextualcompression)
4. [LLM을 활용한 문서 필터링](#llm을-활용한-문서-필터링)
5. [파이프라인 생성](#파이프라인-생성압축기문서-변환기)
6. [요약](#요약)

## 개요

### 문맥 압축 검색기가 필요한 이유

검색 시스템에서 직면하는 주요 문제점:
- 데이터 수집 시점에 어떤 질의가 들어올지 예측 불가
- 관련 정보가 무관한 텍스트에 묻혀있을 수 있음
- 전체 문서 전달 시 비용 증가 및 응답 품질 저하

### ContextualCompressionRetriever의 작동 원리

1. **질의 전달**: Base retriever에 질의를 전달
2. **문서 검색**: 초기 문서 집합 가져오기
3. **문서 압축**: Document Compressor를 통해 관련 정보만 추출
   - 개별 문서 내용 압축
   - 관련 없는 문서 필터링

> 💡 **핵심 아이디어**: 질의의 맥락을 활용하여 관련 정보만 선별적으로 반환

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"문서 {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 기본 Retriever 설정

먼저 일반적인 벡터 스토어 retriever를 설정하고, 압축 없이 검색했을 때의 결과를 확인해봅시다.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# TextLoader를 사용하여 "appendix-keywords.txt" 파일에서 문서를 로드합니다.
loader = TextLoader("../appendix-keywords.txt")

# CharacterTextSplitter를 사용하여 문서를 청크 크기 300자와 청크 간 중복 0으로 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

# OpenAIEmbeddings를 사용하여 FAISS 벡터 저장소를 생성하고 검색기로 변환합니다.
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

# 쿼리에 질문을 정의하고 관련 문서를 검색합니다.
docs = retriever.invoke("Semantic Search 에 대해서 알려줘.")

# 검색된 문서를 예쁘게 출력합니다.
pretty_print_docs(docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

### 관찰 결과

기본 retriever는 4개의 문서를 반환했습니다:
- ✅ **문서 1**: Semantic Search (직접적으로 관련)
- ⚠️ **문서 2-4**: 키워드 검색, 크롤링, 페이지 랭크 (간접적이거나 관련 없음)

이제 ContextualCompressionRetriever를 사용하여 관련성을 개선해봅시다.

## 맥락적 압축 (ContextualCompression)

### LLMChainExtractor

`LLMChainExtractor`는 LLM을 사용하여 문서에서 질의와 관련된 부분만을 추출합니다.

In [4]:
from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain_openai import ChatOpenAI

# OpenAI 언어 모델 초기화
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# LLM을 사용하여 문서 압축기 생성
compressor = LLMChainExtractor.from_llm(llm)

# 문서 압축기와 리트리버를 사용하여 컨텍스트 압축 리트리버 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

# 압축 전 결과
pretty_print_docs(retriever.invoke("Semantic Search 에 대해서 알려줘."))

print("=========================================================")
print("============== LLMChainExtractor 적용 후 ==================")

# 압축 후 결과
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

### 결과 분석

**LLMChainExtractor 효과:**
- 4개 문서 → 1개 문서로 압축
- 질의와 직접 관련된 Semantic Search 정보만 추출
- 불필요한 정보 제거로 응답 품질 향상

## LLM을 활용한 문서 필터링

### LLMChainFilter

`LLMChainFilter`는 문서 내용을 변경하지 않고 전체 문서를 선택적으로 반환합니다.

In [5]:
from langchain_teddynote.document_compressors import LLMChainFilter

# LLM을 사용하여 LLMChainFilter 객체를 생성합니다.
_filter = LLMChainFilter.from_llm(llm)

# LLMChainFilter와 retriever를 사용하여 ContextualCompressionRetriever 객체를 생성합니다.
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter,
    base_retriever=retriever,
)

compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


### LLMChainFilter vs LLMChainExtractor

| 특징 | LLMChainFilter | LLMChainExtractor |
|------|----------------|-------------------|
| 동작 방식 | 문서 전체 반환/제거 | 문서 내 관련 부분만 추출 |
| 문서 내용 변경 | ❌ | ✅ |
| 사용 시기 | 문서 단위 필터링 필요 시 | 문서 내 압축 필요 시 |

### EmbeddingsFilter

`EmbeddingsFilter`는 임베딩 유사도를 기반으로 문서를 필터링합니다.

**장점:**
- ✅ LLM 호출 없이 작동 (빠르고 저렴)
- ✅ 유사도 임계값으로 세밀한 제어 가능

In [6]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# 유사도 임계값이 0.86인 EmbeddingsFilter 객체를 생성합니다.
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings, 
    similarity_threshold=0.86  # 0.86 이상의 유사도를 가진 문서만 반환
)

# 기본 압축기로 embeddings_filter를, 기본 검색기로 retriever를 사용
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, 
    base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


### 유사도 임계값 선택 가이드

- **0.9 이상**: 매우 엄격한 필터링 (거의 완벽한 매칭)
- **0.8 ~ 0.9**: 높은 관련성 요구
- **0.7 ~ 0.8**: 중간 수준의 관련성
- **0.7 이하**: 느슨한 필터링

## 파이프라인 생성(압축기+문서 변환기)

`DocumentCompressorPipeline`을 사용하면 여러 압축기와 변환기를 순차적으로 결합할 수 있습니다.

### 파이프라인 구성 요소

1. **CharacterTextSplitter**: 문서를 작은 청크로 분할
2. **EmbeddingsRedundantFilter**: 중복 문서 제거 (유사도 0.95 이상)
3. **EmbeddingsFilter**: 관련성 기준 필터링
4. **LLMChainExtractor**: 최종 내용 압축

In [7]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

# 1. 문자 기반 텍스트 분할기 생성
splitter = CharacterTextSplitter(
    chunk_size=300,     # 각 청크의 최대 크기
    chunk_overlap=0     # 청크 간 중복 없음
)

# 2. 중복 필터 생성 (임베딩 유사도 0.95 이상 제거)
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 3. 관련성 필터 생성 (유사도 0.86 이상만 통과)
relevant_filter = EmbeddingsFilter(
    embeddings=embeddings, 
    similarity_threshold=0.86
)

# 4. 파이프라인 생성
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[
        splitter,           # 1단계: 문서 분할
        redundant_filter,   # 2단계: 중복 제거
        relevant_filter,    # 3단계: 관련성 필터링
        LLMChainExtractor.from_llm(llm),  # 4단계: 내용 압축
    ]
)

### 파이프라인 처리 흐름

```
원본 문서 → [분할] → 작은 청크들 → [중복 제거] → 고유 청크들 → [관련성 필터] → 관련 청크들 → [내용 압축] → 최종 결과
```

In [8]:
# 파이프라인 압축기를 사용한 ContextualCompressionRetriever 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,
    base_retriever=retriever,
)

# 파이프라인 실행
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)

# 결과 출력
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


## 요약

### 주요 학습 내용

1. **ContextualCompressionRetriever의 필요성**
   - 검색된 문서의 관련 정보만 추출
   - LLM 호출 비용 절감 및 응답 품질 향상

2. **다양한 압축 방법**
   - **LLMChainExtractor**: 문서 내용 압축
   - **LLMChainFilter**: 문서 전체 필터링
   - **EmbeddingsFilter**: 임베딩 기반 빠른 필터링

3. **파이프라인 구성**
   - 여러 압축기와 변환기를 순차적으로 결합
   - 분할 → 중복 제거 → 관련성 필터링 → 내용 압축

### 사용 시나리오별 권장 사항

| 시나리오 | 권장 방법 |
|---------|----------|
| 빠른 필터링 필요 | EmbeddingsFilter |
| 정확한 내용 추출 | LLMChainExtractor |
| 문서 단위 선택 | LLMChainFilter |
| 복잡한 처리 | DocumentCompressorPipeline |

### 성능 vs 품질 트레이드오프

- **EmbeddingsFilter**: 빠르지만 덜 정확
- **LLM 기반 방법**: 느리지만 더 정확
- **파이프라인**: 균형잡힌 접근

### 다음 단계

- 다양한 유사도 임계값 실험
- 커스텀 압축기 구현
- 실제 애플리케이션에 통합